In [1]:
import numpy as np

In [2]:
def mod_pert_random(low, likely, high, confidence=4, samples=30):
    """Produce random numbers according to the 'Modified PERT' 
    distribution. 
    
    :param low: The lowest value expected as possible.
    
    :param likely: The 'most likely' value, statistically, the mode.
    :param high: The highest value expected as possible.
    :param confidence: This is typically called 'lambda' in literature 
                        about the Modified PERT distribution. The value
                        4 here matches the standard PERT curve. Higher
                        values indicate higher confidence in the mode.
                        
    Formulas from "Modified Pert Simulation" by Paulo Buchsbaum.
    """
    # Check minimum & maximum confidence levels to allow:
    confidence = min(8, confidence)
    confidence = max(2, confidence)

    mean = (low + confidence * likely + high)/(confidence + 2)

    a = (mean - low)/(high - low) * (confidence + 2)
    b = ((confidence + 1) * high - low - confidence * likely) / (high - low)
    
    beta = np.random.beta(a, b, samples)
    beta = beta * (high - low) + low
    return beta

In [3]:
class Queue:
    def __init__(self):
        self.groups=[]
    
    def get_groups(self):
        return self.groups
    
    def queue_size(self):
        return len(self.groups)
    
    def isEmpty(self):
        if len(self.groups)> 0:
            return False
        else:
            
            return True
    
    def add_queue(self, group):
        if group.get_vip():   # If current group is a VIP group, move it forward by four groups,
            # but the head of the queue would never be cut
            enterQueue=False
            if len(self.groups) > 4:
                for i in range(0,4):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                        self.groups.insert(4,group)
            elif len(self.groups) > 1 and (len(self.groups) <=4):
                for i in range(0,len(self.groups)):
                    if self.groups[i].get_vip():
                        self.groups.insert(i,group)
                        enterQueue=True
                        break
                if (enterQueue is False):
                    self.groups.insert((len(self.groups)-1),group)
        else: 
            self.groups.insert(0,group)
            
    
    def del_queue(self): #delete last=delete first come group 
        return self.groups.pop()  

In [4]:
# this class can be regarded as "class printer" in that case.
class Table:
    def __init__(self, num, size):
        self.num= num # No. of the table
        self.size = size # Size of the table: for group of up to 2, 4 or 6.
        
        self.currentGroup = None # Is the table occupied or not.
        
    def busy(self):
        if self.currentGroup != None:
            return True
        else:
            return False      
    
    def startNext(self, newGroup):
        self.currentGroup = newGroup

In [5]:
class Group:
    def __init__(self, time, size, vip, groupID):
        self.timestamp = time # Time when group registered (entered into the queue)
        self.size = size #randomly define size from 1 - 6
        self.vip = vip # Whether the group is a vip group
        self.table= None # Which table the group will be assigned to
        self.timeRequest = mod_pert_random(0, 45, 120, samples=1).astype(int)  #How long will the group spend
        self.groupID=groupID
    
    def get_groupID(self):
        return self.groupID
    
    def get_stamp(self):
        return self.timestamp
    
    def get_size(self):
        return self.size
    
    
    def wait_time(self, current_time):
        return current_time - self.timestamp
    
    def get_vip(self):
        return self.vip
    
    def get_time_request(self): 
        return self.timeRequest

In [6]:
def tablesSetting(number_tables_2, number_tables_4, number_tables_6):
    table_2_list = []
    table_4_list = []
    table_6_list = [] 
    for i in range(number_tables_2):
        table_2_list.append(Table(i,2))
    
    for i in range(number_tables_4):
        table_4_list.append(Table(i+number_tables_2,4))
    
    for i in range(number_tables_6):
        table_6_list.append(Table(i+number_tables_4+number_tables_2,6))
    
    return (table_2_list, table_4_list, table_6_list)

In [7]:
def generation(Duration):
    # Generate size 
    size = np.random.randint(1,7,30)
    
    vip = []
    for i in range(len(size)):
        num = np.random.randint(0, 101, 1)
        if (num >= 0) & (num <= 8):
            vip.append(True)
        else:
            vip.append(False)
    timestamp_list = mod_pert_random(0, 120, 240).astype(int)
    
    print(timestamp_list)
    
    counter = 0
    queue_2 = Queue()
    queue_4 = Queue()
    queue_6 = Queue()
    
    
    table_2,table_4,table_6=tablesSetting(6,4,2)
    
    total_timeR = [0]
    nextGroup_endTime_2={}
    
    # Add every group into corresponding queue
    groupNumb_2=0
    groupNumb_4=0
    groupNumb_6=0
    
    for i in range(Duration):
        if i in timestamp_list:
            if size[counter] == 1 or size[counter] == 2:
                queue_2.add_queue(Group(i, 2, vip[counter], groupNumb_2))
                counter += 1
                groupNumb_2+=1
            elif size[counter] == 3 or size[counter] == 4:
                queue_4.add_queue(Group(i, 4, vip[counter], groupNumb_4))
                counter += 1
                groupNumb_4+=1
            elif size[counter] == 5 or size[counter] == 6:
                queue_6.add_queue(Group(i, 6, vip[counter], groupNumb_6))
                counter += 1
                groupNumb_6 += 1
                
                
        if i in nextGroup_endTime_2.values():
            for num in nextGroup_endTime_2.keys():
                if i == nextGroup_endTime_2[num]:
                    print('Clean:', num.num,'\n')
                    table_2[num.num].currentGroup = None
            #if at least one of tables2 finish servicing, set currentGroup to None (this table is not busy)
        if (len(queue_2.groups)>0):
            # print current queue situation
            current_q={}
            for item in queue_2.groups:
                current_q[item.get_groupID()]=[item.get_stamp(), item.get_vip()]
            print(current_q)
            
            if (i >= (queue_2.groups[queue_2.queue_size()-1].get_stamp())):
                for t in table_2:
                    if (not t.busy()) and (i>=(queue_2.groups[queue_2.queue_size()-1].get_stamp())):
                        print('Head GroupID:',queue_2.groups[queue_2.queue_size()-1].get_groupID())
                        nextGroup = queue_2.del_queue()
                     
                        print('Wait time:', nextGroup.wait_time(i))
                     
                     
                     
                        t.startNext(nextGroup)
                        print('Group No.:', nextGroup.get_groupID())
                        print('VIP?:', nextGroup.get_vip())
                        print('Registration time:', nextGroup.get_stamp())
                        print('Entering into table time:',i)
                         
                         
                         
                        nextGroup.table=t.num
                        print('Table No.', t.num)
                         
                        nextGroup_endTime_2[t] = i+nextGroup.timeRequest+2  #2min for table cleaning, rearragement and the time to walk in blabla
                        print('\n')
                        total_timeR.append(nextGroup.timeRequest+2)
        
    avg_timeR = sum(total_timeR)/len(total_timeR)

In [8]:
generation(240)

[162 128 127 160  51 192 113  78 141 101  85 140 186  53 177 121 132 189
 164 174 227 123 160 165  68 168  95  56  70 189]
{0: [56, False]}
Head GroupID: 0
Wait time: 0
Group No.: 0
VIP?: False
Registration time: 56
Entering into table time: 56
Table No. 0




IndexError: list index out of range